## Energy saved from recycling
---

**Project Description**
Did you know that recycling saves energy by reducing or eliminating the need to make materials from scratch? For example, aluminum can manufacturers can skip the energy-costly process of producing aluminum from ore by cleaning and melting recycled cans. Aluminum is classified as a non-ferrous metal.

Singapore has an ambitious goal of becoming a zero-waste nation. The amount of waste disposed of in Singapore has increased seven-fold over the last 40 years. At this rate, Semakau Landfill, Singapore’s only landfill, will run out of space by 2035. Making matters worse, Singapore has limited land for building new incineration plants or landfills.

The government would like to motivate citizens by sharing the total energy that the combined recycling efforts have saved every year. They have asked you to help them.

You have been provided with three datasets. The data come from different teams, so the names of waste types may differ.
<img src="images/waste_stats.png" width="800px"/>

**Question to answer**:<br>
How much energy in kiloWatt hour (kWh) has Singapore saved per year by recycling glass, plastic, ferrous, and non-ferrous metals between 2015 and 2019?

### Table of Contents:
* [Get to know the datasets](#get_to_know)
* [Check for abnormalities](#check)
* [Clean data](#clean)
* [Merge](#merge)
* [Solution](#solution)

### Get to know the datasets <a class="anchor" id="get_to_know"/>
We are given three datasets, two of which share identical columns with minor changes in naming and metrics. First, let's check the content of each dataset, specifically paying attention to unique values, duplications, and missing values. 

From the first sight, we can relate the following columns (left: 2003-2017, right: 2018-2019):<br>
waste_type = Waste Type <br>
total_waste_recycle_tonne = Total Recycled <br>
total_waste_generated = Total Generated <br>
year = Year <br>

Let's check if that's the case!

In [2]:
# import packages
import re
import numpy as np
import pandas as pd

In [3]:
# load data
stats2003_2017 = pd.read_csv('datasets/wastestats.csv')
stats2018_2019 = pd.read_csv('datasets/2018_2019_waste.csv')

# we need to skip the first three rows since it includes description
energy_saved = pd.read_csv('datasets/energy_saved.csv', skiprows=3, index_col=0)

In [4]:
# show first three rows
stats2003_2017.head(3)

,waste_type,waste_disposed_of_tonne,total_waste_recycled_tonne,total_waste_generated_tonne,recycling_rate,year
0,Food,679900,111100.0,791000,0.14,2016
1,Paper/Cardboard,576000,607100.0,1183100,0.51,2016
2,Plastics,762700,59500.0,822200,0.07,2016


In [5]:
# show first three rows
stats2018_2019.head(3)

,Waste Type,Total Generated ('000 tonnes),Total Recycled ('000 tonnes),Year
0,Construction& Demolition,1440,1434,2019
1,Ferrous Metal,1278,1270,2019
2,Paper/Cardboard,1011,449,2019


In [6]:
energy_saved

,Plastic,Glass,Ferrous Metal,Non-Ferrous Metal,Paper
material,,,,,
energy_saved,5774 Kwh,42 Kwh,642 Kwh,14000 Kwh,4000 kWh
crude_oil saved,16 barrels,NaN,1.8 barrels,40 barrels,1.7 barrels


### Check for abnormalities <a class="anchor" id="check"/>
As we can see below, there are no duplications and all columns that need to have numeric values, are actually numeric! However, let's focus on the potential missing values.

In [7]:
# check for duplications
print('# of duplicates in 2003-2017 dataset:', sum(stats2003_2017.duplicated()))
print('# of duplicates in 2018-2019 dataset:', sum(stats2018_2019.duplicated()))

# of duplicates in 2003-2017 dataset: 0
# of duplicates in 2018-2019 dataset: 0


In [8]:
# check for missing values and dtypes
print(stats2003_2017.info())

print('\n=============\n')

print(stats2018_2019.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   waste_type                   225 non-null    object 
 1   waste_disposed_of_tonne      225 non-null    int64  
 2   total_waste_recycled_tonne   225 non-null    float64
 3   total_waste_generated_tonne  225 non-null    int64  
 4   recycling_rate               225 non-null    float64
 5   year                         225 non-null    int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 10.7+ KB
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Waste Type                     30 non-null     object
 1   Total Generated ('000 tonnes)  30 non-null     int64 
 2   Total Recycled ('00

In [9]:
# select only numeric columns and check how many of them are 0
only_ints2003_2017 = stats2003_2017.select_dtypes(include=['int', 'float'])
only_ints2018_2019 = stats2018_2019.select_dtypes(include=['int', 'float'])

# show how many values are 0
print('# of rows with 0 values:', np.sum(only_ints2003_2017 == 0))
print('# of rows with 0 values:', np.sum(only_ints2018_2019 == 0))

# of rows with 0 values: total_waste_recycled_tonne    11
recycling_rate                11
dtype: int64
# of rows with 0 values: Series([], dtype: float64)


#### Closer look
We saw an interesting finding above, there are eleven 0 values in both total_waste_recycled_tonne and recycling_rate. I wonder if it is due to non-recyclable waste types. Furthermore, let's check if total generated column is equal to total disposed ones. If they're equal, then the "missing" values are actually justified. 

In [10]:
# look closer at 0 values 
missing = stats2003_2017[(stats2003_2017['total_waste_recycled_tonne'] == 0) | 
                         (stats2003_2017['recycling_rate'] == 0)]
missing

,waste_type,waste_disposed_of_tonne,total_waste_recycled_tonne,total_waste_generated_tonne,recycling_rate,year
69,Sludge,160200,0.0,160200,0.0,2012
84,Sludge,152900,0.0,152900,0.0,2011
99,Sludge,114400,0.0,114400,0.0,2010
114,Sludge,88900,0.0,88900,0.0,2009
129,Sludge,114600,0.0,114600,0.0,2008
144,Sludge,124600,0.0,124600,0.0,2007
159,Sludge,126800,0.0,126800,0.0,2006
174,Sludge,98300,0.0,98300,0.0,2005
189,Sludge,93900,0.0,93900,0.0,2004
204,Sludge,88500,0.0,88500,0.0,2003


In [11]:
# verify disposed waste == total waste generated
assert np.all(missing['total_waste_recycled_tonne'] == missing['recycling_rate'])

#### Success!
Indeed, all of the zero-valued rows cannot be categorized as missing values! From what we see above, sludge is the most occuring waste type that could not be recycled until 2012. Let's see the progression in sludge recycling over the next years after they learned how to handle it.

In [12]:
stats2003_2017[stats2003_2017['waste_type'] == 'Sludge'][['year', 'recycling_rate']]

,year,recycling_rate
54,2013,0.07
69,2012,0.00
84,2011,0.00
99,2010,0.00
114,2009,0.00
129,2008,0.00
144,2007,0.00
159,2006,0.00
174,2005,0.00
189,2004,0.00


### Clean Data <a class="anchor" id="clean"/>

**Structures** of the 2003-2017 and 2018-2019 are pretty much the same. However, if we look at unique values, we see the first has duplications. Thus, we need to clean out the names, put them on the same scale and prepare for merging datasets.

In [7]:
unique_waste2003_2017 = stats2003_2017['waste_type'].unique()
unique_waste2018_2019 = stats2018_2019['Waste Type'].unique()

print(f'2003-2017 has {len(unique_waste2003_2017)} types of waste:\n', unique_waste2003_2017)
print('\n=============\n')
print(f'2018-2019 has {len(unique_waste2018_2019)} types of waste:\n', unique_waste2018_2019)


2003-2017 has 32 types of waste:
 ['Food' 'Paper/Cardboard' 'Plastics' 'C&D' 'Horticultural waste' 'Wood'
 'Ferrous metal' 'Non-ferrous metal' 'Used slag' 'Ash & Sludge' 'Glass'
 'Textile/Leather' 'Scrap tyres' 'Others (stones, ceramics & rubber etc.)'
 'Total' 'Others (stones, ceramics & rubber etc)' 'Food waste'
 'Construction Debris' 'Wood/Timber' 'Horticultural Waste' 'Ferrous Metal'
 'Non-ferrous Metals' 'Used Slag' 'Sludge' 'Scrap Tyres' 'Ferrous Metals'
 'Others' 'Construction debris' 'Non-ferrous metals' 'Ash and sludge'
 'Plastic' 'Others (stones, ceramic, rubber, etc.)']


2018-2019 has 15 types of waste:
 ['Construction& Demolition' 'Ferrous Metal' 'Paper/Cardboard' 'Plastics'
 'Food' 'Wood' 'Horticultural' 'Ash & Sludge' 'Textile/Leather'
 'Used Slag' 'Non-Ferrous Metal' 'Glass' 'Scrap Tyres'
 'Others (stones, ceramic, rubber, ect)' 'Overall']


In [8]:
# print what they have in common
print('Common:')
print(list(set(unique_waste2003_2017).intersection(set(unique_waste2018_2019))))

print('\n=============\n')

# print how they differ
print('Difference:')
print(list(set(unique_waste2003_2017).difference(set(unique_waste2018_2019))))


Common:
['Glass', 'Used Slag', 'Scrap Tyres', 'Food', 'Wood', 'Ferrous Metal', 'Ash & Sludge', 'Paper/Cardboard', 'Plastics', 'Textile/Leather']


Difference:
['Non-ferrous Metals', 'Others', 'C&D', 'Sludge', 'Scrap tyres', 'Ash and sludge', 'Plastic', 'Non-ferrous metal', 'Construction Debris', 'Non-ferrous metals', 'Wood/Timber', 'Horticultural waste', 'Total', 'Food waste', 'Others (stones, ceramics & rubber etc)', 'Construction debris', 'Horticultural Waste', 'Others (stones, ceramic, rubber, etc.)', 'Others (stones, ceramics & rubber etc.)', 'Used slag', 'Ferrous Metals', 'Ferrous metal']


In [13]:
def replace(name):    
    """
    This function cleans up the waste types' names. 
        name: Name that needs to be taken care of
    returns: Prettified representation of the class name
    """
    name = re.sub('[\.]+', '', name.lower())
    name = name.replace('  ', ' ').replace('and', '&').replace('overall', 'total')
    if name.startswith('construction') or name.startswith('c&d'):
        name = 'construction & demolition'
    elif name.startswith('others'):
        name = 'others (stones, ceramics & rubber etc.)'
    elif name.startswith('plastic'):
        name = 'plastic'
    elif name.startswith('horticultural'):
        name = 'horticultural'
    elif name.startswith('non-ferrous'):
        name = 'non-ferrous metal'
    elif name.startswith('food'):
        name = 'food'
    elif name.startswith('ferrous'):
        name = 'ferrous metal'
    elif name.startswith('wood'):
        name = 'wood'
    elif name.startswith('sludge'):
        name = 'ash & sludge'
    
    return name.title()


# apply transformations
stats2003_2017['waste_type'] = stats2003_2017['waste_type'].apply(replace)
stats2018_2019['Waste Type'] = stats2018_2019['Waste Type'].apply(replace)

# get unique values once again
unique_waste2003_2017 = stats2003_2017['waste_type'].unique()
unique_waste2018_2019 = stats2018_2019['Waste Type'].unique()

# verify there is no difference in waste types 
assert set(unique_waste2003_2017) == set(unique_waste2018_2019) 

### Merge <a class="anchor" id="merge"/>
Everything is taken care of! Now, we need to change column names and alter 2018-2019 dataset, and put them both on the same scale. 

In [15]:
COLUMNS = ['Waste Type', 'Total Disposed (thousands)', 'Total Recycled (thousands)',
                'Total Generated (thousands)', 'Recycling Rate', 'year']

# change 2003-2017 dataset's columns for further merging 
stats2003_2017.columns = COLUMNS
stats2003_2017[['Total Disposed (thousands)', 'Total Recycled (thousands)', 'Total Generated (thousands)']] /= 1000

# check first 5 entries
stats2003_2017.head()

,Waste Type,Total Disposed (thousands),Total Recycled (thousands),Total Generated (thousands),Recycling Rate,year
0,Food,679.9,111.1,791.0,0.14,2016
1,Paper/Cardboard,576.0,607.1,1183.1,0.51,2016
2,Plastic,762.7,59.5,822.2,0.07,2016
3,Construction & Demolition,9.7,1585.7,1595.4,0.99,2016
4,Horticultural,111.5,209.0,320.5,0.65,2016


In [16]:
disposed = stats2018_2019['Total Generated (\'000 tonnes)'] - stats2018_2019['Total Recycled (\'000 tonnes)']
recycling_rate = round(stats2018_2019['Total Recycled (\'000 tonnes)'] / stats2018_2019['Total Generated (\'000 tonnes)'], 3)

# add missing columns 
stats2018_2019['Total Disposed (thousands)'] = disposed
stats2018_2019['Recycling Rate'] = recycling_rate
stats2018_2019.head()

,Waste Type,Total Generated ('000 tonnes),Total Recycled ('000 tonnes),Year,Total Disposed (thousands),Recycling Rate
0,Construction & Demolition,1440,1434,2019,6,0.996
1,Ferrous Metal,1278,1270,2019,8,0.994
2,Paper/Cardboard,1011,449,2019,562,0.444
3,Plastic,930,37,2019,893,0.040
4,Food,7440,136,2019,7304,0.018


In [17]:
# change column order for 2018-2019 dataset
stats2018_2019 = stats2018_2019[['Waste Type', 'Total Disposed (thousands)', 'Total Recycled (\'000 tonnes)',
                'Total Generated (\'000 tonnes)', 'Recycling Rate', 'Year']]

# change column names 
stats2018_2019.columns = COLUMNS
stats2018_2019.head()

,Waste Type,Total Disposed (thousands),Total Recycled (thousands),Total Generated (thousands),Recycling Rate,year
0,Construction & Demolition,6,1434,1440,0.996,2019
1,Ferrous Metal,8,1270,1278,0.994,2019
2,Paper/Cardboard,562,449,1011,0.444,2019
3,Plastic,893,37,930,0.040,2019
4,Food,7304,136,7440,0.018,2019


In [18]:
# merge datasets
all_years = pd.concat([stats2003_2017, stats2018_2019], ignore_index=True)

# verify the length of the new dataset
assert len(all_years) == len(stats2003_2017) + len(stats2018_2019)

In [19]:
# select columns that are needed for the assignment
COLUMNS = ['Glass', 'Plastic', 'Ferrous Metal', 'Non-Ferrous Metal']

# get specific waste types column starting from 2015
all_years = all_years[all_years['year'] > 2014][['Waste Type', 'Total Recycled (thousands)', 'year']]

# get only waste types that are specified above
all_years = all_years[all_years['Waste Type'].isin(COLUMNS)]

# show the dataset
all_years

,Waste Type,Total Recycled (thousands),year
2,Plastic,59.5,2016
6,Ferrous Metal,1351.5,2016
7,Non-Ferrous Metal,95.9,2016
10,Glass,14.7,2016
17,Plastic,57.8,2015
21,Ferrous Metal,1333.3,2015
22,Non-Ferrous Metal,160.4,2015
25,Glass,14.6,2015
211,Ferrous Metal,1371.0,2017
213,Non-Ferrous Metal,92.2,2017


### Solution <a class="anchor" id="solution"/>

In "energy_saved.csv" we have the information about energy savings. According to the assignment, we are interested in kWh representation. Let's look at how much energy is saved per recycling 1 metric tonne. After that, we need to calculate each waste type and store the results in *annual_energy_savings*.


In [20]:
# check the kwh distribution 
energy_saved.iloc[0, :]

Plastic               5774 Kwh
Glass                   42 Kwh
Ferrous Metal          642 Kwh
Non-Ferrous Metal    14000 Kwh
Paper                 4000 kWh
Name: energy_saved, dtype: object

In [21]:
def calculate(waste_type, recycled):
    """
    This function calculates energy savings per waste types according to "energy_saved.csv".
        waste_type (str):      Type of waste
        recycled (int, float): Number of recycled tonnes a year
    return: Recycled calculation per waste type
    """
    # calculate waste type 
    if waste_type == 'Plastic':
        return recycled * 5774 
    elif waste_type == 'Glass':
        return recycled * 42
    elif waste_type == 'Ferrous Metal':
        return recycled * 642 
    return recycled * 14000

In [22]:
# calculate energy saved and add a new column for that 
total = all_years.apply(lambda x: calculate(x['Waste Type'], 
                                            x['Total Recycled (thousands)']), axis=1)
# add a new column to store calculations
all_years['total_energy_saved'] = total

In [23]:
# create the final dataset to store each year's energy savings
annual_energy_savings = all_years.groupby('year').agg({'total_energy_saved': np.sum})
annual_energy_savings['total_energy_saved'] *= 1000
annual_energy_savings

,total_energy_saved
year,
2015,3.435929e+09
2016,2.554433e+09
2017,2.470596e+09
2018,2.698130e+09
2019,2.765440e+09
